In [14]:
import pandas as pd

output_csv = r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\7rada\Catalog_products_patchi.xlsx"
pat =r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\cleaned_sheets\NO ADDED SUGAR_cleaned.xlsx"
# Lire toutes les feuilles
df = pd.read_excel(pat) 
df = df.dropna(axis=1, how='all')  # drop columns where all values are NaN
print(df.columns)




Index([0, 1, 2, 3, 4, 5], dtype='int64')


In [2]:
import pandas as pd

output_csv = r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\7rada\feuilles_extraites\NO ADDED SUGAR.xlsx"
df = pd.read_excel(output_csv) 
print(df.columns)


Index(['codes ', 'Name ', 'new  Photo ', 'filling', 'Weight ', 'Price'], dtype='object')


In [11]:
import pandas as pd
import os
from pathlib import Path

# Path to final merged catalog
final_path = r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\7rada\Catalog_products_patchi.xlsx"

# Load the final catalog (must exist)
final_df = pd.read_excel(final_path)

print("=== Final Catalog Columns ===")
print(final_df.columns.tolist())
print(f"Current products in catalog: {len(final_df)}")

# Directory containing extracted sheets
sheets_dir = Path(r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\7rada\feuilles_extraites")

# Counter for total products added
total_added = 0

# Helper function to find column by possible names (case-insensitive)
def find_column(df, possible_names):
    """Find a column in dataframe by checking multiple possible names (case-insensitive)"""
    df_cols_lower = {col.lower(): col for col in df.columns}
    
    for name in possible_names:
        if name.lower() in df_cols_lower:
            return df_cols_lower[name.lower()]
    return None

# Process each Excel file in the directory
for sheet_file in sheets_dir.glob("*.xlsx"):
    # Skip the main catalog file itself
    if sheet_file.name == "Catalog_products_patchi.xlsx":
        print(f"\n⊗ Skipping main catalog file: {sheet_file.name}")
        continue
    
    print(f"\n=== Processing: {sheet_file.name} ===")
    
    try:
        # Load the extracted product sheet
        df = pd.read_excel(sheet_file)
        
        # Clean column names (remove trailing/leading spaces)
        df.columns = df.columns.str.strip()
        
        print(f"Sheet columns: {df.columns.tolist()}")
        print(f"Products found: {len(df)}")
        
        # Find columns with flexible matching
        col_code = find_column(df, ["codes", "code", "codes ", "Code", "Codes"])
        col_name = find_column(df, ["name", "Name", "name ", "Name "])
        col_price = find_column(df, ["price", "Price", "prix", "Prix", "price ", "Price "])
        col_photo = find_column(df, ["new photo", "new  photo", "photo", "Photo", "New Photo"])
        col_filling = find_column(df, ["filling", "Filling", "Final fillin"])
        col_weight = find_column(df, ["weight", "Weight", "weight ", "Weight ", "Final Weight", "Final weiht"])
        
        # Check if essential columns exist
        if not col_code:
            print(f"⚠ Missing 'codes' column (tried: codes, code, Code, Codes)")
            print("Skipping this sheet...")
            continue
        
        if not col_name:
            print(f"⚠ Missing 'name' column (tried: name, Name)")
            print("Skipping this sheet...")
            continue
        
        if not col_price:
            print(f"⚠ Missing 'price' column (tried: price, Price, prix, Prix)")
            print("Skipping this sheet...")
            continue
        
        # Check for empty IDs
        empty_ids = df[col_code].isna().sum()
        if empty_ids > 0:
            print(f"⚠ Found {empty_ids} products with missing IDs - removing them")
            df = df[df[col_code].notna()]
        
        if len(df) == 0:
            print("⚠ No valid products left after filtering")
            continue
        
        # Use filename as brand name (without extension)
        brand_name = sheet_file.stem
        
        # Build the dataframe with ALL required columns
        mapped_df = pd.DataFrame({
            "ordering_index": range(len(final_df), len(final_df) + len(df)),
            "id": df[col_code],
            "title": df[col_name],
            "price": df[col_price],
            "description": df[col_filling] if col_filling else "",
            "availability": "in stock",
            "condition": "new",
            "link": "",
            "image_link": df[col_photo] if col_photo else "",
            "additional_image_link": "",
            "brand": brand_name,
            "google_product_category": "",
            "fb_product_category": "",
            "quantity_to_sell_on_facebook": "",
            "sale_price": "",
            "sale_price_effective_date": "",
            "item_group_id": "",
            "gender": "",
            "color": "",
            "size": "",
            "age_group": "",
            "material": "",
            "pattern": "",
            "shipping": "",
            "shipping_weight": df[col_weight] if col_weight else "",
            "gtin": "",
            "video[0].url": "",
            "video[0].tag[0]": "",
            "product_tags[0]": "",
            "product_tags[1]": "",
            "style[0]": "",
            "retailer_id": ""
        })
        
        # Append to final catalog
        final_df = pd.concat([final_df, mapped_df], ignore_index=True)
        
        total_added += len(mapped_df)
        print(f"✓ Added {len(mapped_df)} products from {brand_name}")
        
    except Exception as e:
        print(f"❌ Error processing {sheet_file.name}: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

# Save updated catalog
final_df.to_excel(final_path, index=False)

print("\n" + "="*50)
print(f"✓ COMPLETED!")
print(f"Total products added: {total_added}")
print(f"Final catalog now has: {len(final_df)} products")
print(f"Saved to: {final_path}")
print("="*50)

=== Final Catalog Columns ===
['ordering_index', 'id', 'title', 'price', 'description', 'availability', 'condition', 'link', 'image_link', 'additional_image_link', 'brand', 'google_product_category', 'fb_product_category', 'quantity_to_sell_on_facebook', 'sale_price', 'sale_price_effective_date', 'item_group_id', 'gender', 'color', 'size', 'age_group', 'material', 'pattern', 'shipping', 'shipping_weight', 'gtin', 'video[0].url', 'video[0].tag[0]', 'product_tags[0]', 'product_tags[1]', 'style[0]', 'retailer_id']
Current products in catalog: 471

=== Processing:  MY Treats.xlsx ===
Sheet columns: ['codes', 'name', 'New Photo', "What's Inside Photo", 'filling', 'Final Weight', 'Price', 'Unnamed: 7', 'Unnamed: 8']
Products found: 22
✓ Added 22 products from  MY Treats

⊗ Skipping main catalog file: Catalog_products_patchi.xlsx

=== Processing: CHOCOLATE BARS.xlsx ===
Sheet columns: ['Code', 'Name', 'Photo', "What's Inside Photo", 'weight', 'ORDER']
Products found: 15
⚠ Missing 'price' colu

In [12]:
import pandas as pd
import openpyxl
from pathlib import Path

def find_header_row(df):
    """
    Détecte la première ligne non vide qui semble être un en-tête.
    Retourne l'index de la ligne d'en-tête.
    """
    for idx, row in df.iterrows():
        # Compte le nombre de valeurs non-nulles dans la ligne
        non_null_count = row.notna().sum()
        
        # Si au moins 50% des colonnes ont des valeurs, c'est probablement l'en-tête
        if non_null_count >= len(row) * 0.5:
            return idx
    
    return 0  # Par défaut, première ligne

def clean_dataframe(df):
    """
    Nettoie le DataFrame en:
    - Détectant l'en-tête réel
    - Supprimant les lignes vides
    - Supprimant les colonnes entièrement vides
    - Réinitialisant l'index
    """
    # Trouve la ligne d'en-tête
    header_idx = find_header_row(df)
    
    # Si l'en-tête n'est pas à la première ligne, réorganise le DataFrame
    if header_idx > 0:
        # Utilise la ligne trouvée comme en-têtes
        new_headers = df.iloc[header_idx].fillna('Unnamed')
        df = df.iloc[header_idx + 1:].copy()
        df.columns = new_headers
    
    # Supprime les lignes entièrement vides
    df = df.dropna(how='all')
    
    # Supprime les colonnes entièrement vides
    df = df.dropna(axis=1, how='all')
    
    # Réinitialise l'index
    df = df.reset_index(drop=True)
    
    # Nettoie les noms de colonnes (supprime espaces superflus)
    df.columns = df.columns.str.strip() if df.columns.dtype == 'object' else df.columns
    
    return df

def process_excel_file(input_file, output_dir='cleaned_sheets'):
    """
    Traite un fichier Excel avec plusieurs feuilles:
    - Lit toutes les feuilles
    - Nettoie chaque feuille
    - Sauvegarde chaque feuille dans un fichier Excel séparé
    
    Args:
        input_file (str): Chemin vers le fichier Excel d'entrée
        output_dir (str): Dossier de sortie pour les fichiers nettoyés
    """
    # Crée le dossier de sortie s'il n'existe pas
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    print(f"Lecture du fichier: {input_file}")
    
    # Lit toutes les feuilles du fichier Excel
    excel_file = pd.ExcelFile(input_file)
    sheet_names = excel_file.sheet_names
    
    print(f"Nombre de feuilles trouvées: {len(sheet_names)}")
    print(f"Noms des feuilles: {sheet_names}\n")
    
    # Traite chaque feuille
    for sheet_name in sheet_names:
        print(f"Traitement de la feuille: '{sheet_name}'")
        
        # Lit la feuille sans en-tête prédéfini
        df = pd.read_excel(input_file, sheet_name=sheet_name, header=None)
        
        print(f"  - Dimensions originales: {df.shape}")
        
        # Nettoie le DataFrame
        df_cleaned = clean_dataframe(df)
        
        print(f"  - Dimensions après nettoyage: {df_cleaned.shape}")
        
        # Crée un nom de fichier sécurisé
        safe_sheet_name = "".join(c if c.isalnum() or c in (' ', '_', '-') else '_' 
                                   for c in sheet_name)
        output_file = output_path / f"{safe_sheet_name}_cleaned.xlsx"
        
        # Sauvegarde dans un nouveau fichier Excel
        df_cleaned.to_excel(output_file, index=False, engine='openpyxl')
        
        print(f"  ✓ Sauvegardé dans: {output_file}\n")
    
    print(f"Traitement terminé! {len(sheet_names)} fichiers créés dans '{output_dir}/'")

# Exemple d'utilisation
if __name__ == "__main__":
    # Remplace par le chemin de ton fichier Excel
    input_excel = r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\7rada\Dashysk.xlsx"
    
    # Lance le traitement
    process_excel_file(input_excel, output_dir='cleaned_sheets')
    
    # Si tu veux traiter un fichier spécifique
    # process_excel_file("mon_fichier.xlsx", output_dir="resultats")

Lecture du fichier: C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\7rada\Dashysk.xlsx
Nombre de feuilles trouvées: 7
Noms des feuilles: ['HOME SPECIAL', 'SMALL BOXES', 'MINIS ', ' MY Treats', 'NO ADDED SUGAR', 'CHOCOLATE BARS', 'GIFT BOXES']

Traitement de la feuille: 'HOME SPECIAL'
  - Dimensions originales: (5, 11)
  - Dimensions après nettoyage: (5, 11)
  ✓ Sauvegardé dans: cleaned_sheets\HOME SPECIAL_cleaned.xlsx

Traitement de la feuille: 'SMALL BOXES'
  - Dimensions originales: (27, 7)
  - Dimensions après nettoyage: (24, 5)
  ✓ Sauvegardé dans: cleaned_sheets\SMALL BOXES_cleaned.xlsx

Traitement de la feuille: 'MINIS '
  - Dimensions originales: (6, 7)
  - Dimensions après nettoyage: (3, 5)
  ✓ Sauvegardé dans: cleaned_sheets\MINIS _cleaned.xlsx

Traitement de la feuille: ' MY Treats'
  - Dimensions originales: (23, 9)
  - Dimensions après nettoyage: (23, 8)
  ✓ Sauvegardé dans: cleaned_sheets\ MY Treats_cleaned.xlsx

Traitement de la feuille: 'NO ADDED SUGAR'
  - Dime

In [28]:
import pandas as pd
import os

folder_path = r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\cleaned_sheets"

for filename in os.listdir(folder_path):
    if filename.endswith(".xlsx") or filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        
        # Lire le fichier
        if filename.endswith(".xlsx"):
            df = pd.read_excel(file_path, header=None)  # lire sans header pour vérifier
        else:
            df = pd.read_csv(file_path, header=None)
        
        # Vérifier si les colonnes sont des nombres (0,1,2…)
        if all(isinstance(col, int) for col in df.iloc[0]):
            # Si oui, prendre la vraie ligne d'en-tête (ligne 2)
            if filename.endswith(".xlsx"):
                df = pd.read_excel(file_path, header=1)
            else:
                df = pd.read_csv(file_path, header=1)
            print(f"✅ Fixed header for {filename}")
        else:
            # Sinon, garder les colonnes existantes
            df.columns = df.iloc[0]  # mettre la première ligne comme nom de colonnes
            df = df[1:]  # supprimer cette ligne de données
            print(f"✅ Header already correct for {filename}")
        
        # Afficher les colonnes
        print(f"File: {filename}")
        print("Columns:", df.columns.tolist())
        print("-" * 40)


✅ Fixed header for  MY Treats_cleaned.xlsx
File:  MY Treats_cleaned.xlsx
Columns: ['id', 'title', 'New Photo ', "What's Inside Photo ", 'description', 'Final Weight ', 'price', 'Unnamed: 7']
----------------------------------------
✅ Fixed header for CHOCOLATE BARS_cleaned.xlsx
File: CHOCOLATE BARS_cleaned.xlsx
Columns: ['id', 'title', 'Photo', 'description', 'weight', 'price']
----------------------------------------
✅ Fixed header for GIFT BOXES_cleaned.xlsx
File: GIFT BOXES_cleaned.xlsx
Columns: ['id', 'title', 'New Photo', "What's Inside Photo ", 'description', 'Final weiht ', 'nb of pcs', 'price']
----------------------------------------
✅ Fixed header for HOME SPECIAL_cleaned.xlsx
File: HOME SPECIAL_cleaned.xlsx
Columns: ['id', 'title', 'Old Photo', 'New Photo', "What's Inside Photo ", 'description', 'weight ', 'nb of pcs', 'shelf life', 'Price $', 'price']
----------------------------------------
✅ Header already correct for MINIS _cleaned.xlsx
File: MINIS _cleaned.xlsx
Columns:

In [30]:
import pandas as pd


# Path to the new Excel file
new_file = r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\Catalog_products_columns_only.xlsx"

# Lire le fichier original pour récupérer les colonnes
df = pd.read_excel(new_file)


print(df.columns)


Index(['ordering_index', 'id', 'title', 'price', 'description', 'availability',
       'condition', 'link', 'image_link', 'additional_image_link', 'brand',
       'google_product_category', 'fb_product_category',
       'quantity_to_sell_on_facebook', 'sale_price',
       'sale_price_effective_date', 'item_group_id', 'gender', 'color', 'size',
       'age_group', 'material', 'pattern', 'shipping', 'shipping_weight',
       'gtin', 'video[0].url', 'video[0].tag[0]', 'product_tags[0]',
       'product_tags[1]', 'style[0]', 'retailer_id'],
      dtype='object')


In [32]:
import pandas as pd
import os
from pathlib import Path

def merge_product_catalogs(folder_path, output_file):
    """
    Fusionne plusieurs fichiers Excel/CSV en un seul catalogue produits
    avec des colonnes standardisées.
    
    Args:
        folder_path (str): Chemin du dossier contenant les fichiers
        output_file (str): Chemin du fichier de sortie
    """
    
    # Colonnes finales standardisées
    final_columns = [
        'ordering_index', 'id', 'title', 'price', 'description', 'availability', 
        'condition', 'link', 'image_link', 'additional_image_link', 'brand',
        'google_product_category', 'fb_product_category',
        'quantity_to_sell_on_facebook', 'sale_price',
        'sale_price_effective_date', 'item_group_id', 'gender', 'color', 'size',
        'age_group', 'material', 'pattern', 'shipping', 'shipping_weight',
        'gtin', 'video[0].url', 'video[0].tag[0]', 'product_tags[0]',
        'product_tags[1]', 'style[0]', 'retailer_id'
    ]
    
    # Liste pour stocker tous les DataFrames
    all_dataframes = []
    
    # Compteurs
    files_processed = 0
    files_skipped = 0
    total_products = 0
    
    print(f"📂 Lecture des fichiers depuis: {folder_path}\n")
    
    # Parcourir tous les fichiers du dossier
    for filename in sorted(os.listdir(folder_path)):
        if not (filename.endswith('.xlsx') or filename.endswith('.csv')):
            continue
        
        file_path = os.path.join(folder_path, filename)
        
        try:
            # Lire le fichier
            print(f"📄 Traitement de: {filename}")
            
            if filename.endswith('.xlsx'):
                # Essayer d'abord sans header spécifique
                df = pd.read_excel(file_path)
                
                # Si la première ligne semble être un header numérique, skip
                if df.columns[0] == 0 or str(df.columns[0]).isdigit():
                    df = pd.read_excel(file_path, header=1)
            else:
                df = pd.read_csv(file_path)
            
            # Nettoyer les noms de colonnes
            df.columns = df.columns.str.strip().str.lower()
            
            # Vérifier les colonnes essentielles
            required_cols = ['id', 'title', 'description', 'price']
            missing_cols = [col for col in required_cols if col not in df.columns]
            
            if len(missing_cols) >= 3:  # Si plus de 3 colonnes manquent
                print(f"   ⚠️  Ignoré - colonnes manquantes: {missing_cols}\n")
                files_skipped += 1
                continue
            
            # Extraire le nom de marque depuis le nom du fichier
            brand_name = filename.replace('_cleaned.xlsx', '').replace('.xlsx', '').replace('_cleaned', '')
            
            # Créer le DataFrame mappé avec les colonnes standardisées
            mapped_data = {
                'id': df.get('id', ''),
                'title': df.get('title', df.get('product_name', df.get('name', ''))),
                'price': df.get('price', df.get('prix', '')),
                'description': df.get('description', df.get('desc', '')),
                'availability': 'in stock',
                'condition': 'new',
                'link': df.get('link', df.get('url', '')),
                'image_link': df.get('image_link', df.get('image', '')),
                'additional_image_link': df.get('additional_image_link', ''),
                'brand': brand_name,
                'google_product_category': df.get('google_product_category', ''),
                'fb_product_category': df.get('fb_product_category', ''),
                'quantity_to_sell_on_facebook': df.get('quantity_to_sell_on_facebook', ''),
                'sale_price': df.get('sale_price', ''),
                'sale_price_effective_date': df.get('sale_price_effective_date', ''),
                'item_group_id': df.get('item_group_id', ''),
                'gender': df.get('gender', ''),
                'color': df.get('color', df.get('couleur', '')),
                'size': df.get('size', df.get('taille', '')),
                'age_group': df.get('age_group', ''),
                'material': df.get('material', df.get('matiere', '')),
                'pattern': df.get('pattern', ''),
                'shipping': df.get('shipping', ''),
                'shipping_weight': df.get('shipping_weight', ''),
                'gtin': df.get('gtin', ''),
                'video[0].url': df.get('video[0].url', ''),
                'video[0].tag[0]': df.get('video[0].tag[0]', ''),
                'product_tags[0]': df.get('product_tags[0]', ''),
                'product_tags[1]': df.get('product_tags[1]', ''),
                'style[0]': df.get('style[0]', ''),
                'retailer_id': df.get('retailer_id', '')
            }
            
            mapped_df = pd.DataFrame(mapped_data)
            
            # Ajouter à la liste
            all_dataframes.append(mapped_df)
            
            files_processed += 1
            total_products += len(mapped_df)
            
            print(f"   ✓ {len(mapped_df)} produits ajoutés")
            print(f"   Colonnes trouvées: {list(df.columns)}\n")
            
        except Exception as e:
            print(f"   ❌ Erreur: {str(e)}\n")
            files_skipped += 1
            continue
    
    # Fusionner tous les DataFrames
    if not all_dataframes:
        print("❌ Aucun fichier valide trouvé!")
        return
    
    print(f"🔗 Fusion de {len(all_dataframes)} fichiers...")
    final_df = pd.concat(all_dataframes, ignore_index=True)
    
    # Ajouter l'index d'ordonnancement
    final_df.insert(0, 'ordering_index', range(len(final_df)))
    
    # Réorganiser les colonnes selon l'ordre final
    final_df = final_df[final_columns]
    
    # Sauvegarder
    print(f"💾 Sauvegarde dans: {output_file}")
    final_df.to_excel(output_file, index=False, engine='openpyxl')
    
    # Résumé final
    print(f"\n{'='*60}")
    print(f"✅ FUSION TERMINÉE!")
    print(f"{'='*60}")
    print(f"📊 Fichiers traités: {files_processed}")
    print(f"⚠️  Fichiers ignorés: {files_skipped}")
    print(f"🛍️  Total produits: {total_products}")
    print(f"📁 Fichier créé: {output_file}")
    print(f"{'='*60}\n")
    
    # Aperçu des marques
    print("📈 Répartition par marque:")
    print(final_df['brand'].value_counts().to_string())
    
    return final_df


# Exemple d'utilisation
if __name__ == "__main__":
    # Configuration
    folder_path = r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\cleaned_sheets"
    output_file = r"C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\Catalog_products_columns_only.xlsx"
    
    # Lancer la fusion
    df_result = merge_product_catalogs(folder_path, output_file)
    
    # Afficher un aperçu
    if df_result is not None:
        print("\n👀 Aperçu des 5 premières lignes:")
        print(df_result.head())

📂 Lecture des fichiers depuis: C:\Users\aamir\Desktop\YC\P\spam-detection-nlp\notebooks\cleaned_sheets

📄 Traitement de:  MY Treats_cleaned.xlsx
   ✓ 22 produits ajoutés
   Colonnes trouvées: ['id', 'title', 'new photo', "what's inside photo", 'description', 'final weight', 'price', 'unnamed: 7']

📄 Traitement de: CHOCOLATE BARS_cleaned.xlsx
   ✓ 15 produits ajoutés
   Colonnes trouvées: ['id', 'title', 'photo', 'description', 'weight', 'price']

📄 Traitement de: GIFT BOXES_cleaned.xlsx
   ✓ 13 produits ajoutés
   Colonnes trouvées: ['id', 'title', 'new photo', "what's inside photo", 'description', 'final weiht', 'nb of pcs', 'price']

📄 Traitement de: HOME SPECIAL_cleaned.xlsx
   ✓ 4 produits ajoutés
   Colonnes trouvées: ['id', 'title', 'old photo', 'new photo', "what's inside photo", 'description', 'weight', 'nb of pcs', 'shelf life', 'price $', 'price']

📄 Traitement de: MINIS _cleaned.xlsx
   ✓ 3 produits ajoutés
   Colonnes trouvées: ['id', 'title', 'description', 'final weight',